In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


train_df = pd.read_csv("/Users/haderie/Downloads/housing/train.csv")
train_df = train_df.drop(columns=[ "zipcode"])

X_train = train_df.drop(columns=["price"]) # features
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
y_train = train_df["price"] / 1000 # target


test_df = pd.read_csv("/Users/haderie/Downloads/housing/test.csv")
test_df = test_df.drop(columns=["id", "date", "zipcode"])

X_test = test_df.drop(columns=["price"]) # featurs

y_test = test_df["price"] / 1000  # target
X_test_scaled = scaler.transform(X_test)



- Consider a feature $X$, a response variable $Y$, and $N$ samples of training data. Implement a polynomial regression model that fits a polynomial of degree $p$ to the data using the least-square method. Use your own implementation from Problem 3 and adapt it for polynomial
regression. If $p=2$, the model will use two features ($X$ and $X^2$), if $p=3$ the model will use 3 features ($X,X^2,X^3$), and so on for larger values of $p$.


In [19]:
def poly_design_matrix(x, p):
    """
    p - polynomial degree
    returns: (N, p+1) design matrix [1, x, x^2, ..., x^p]
    """
    return np.column_stack([x**i for i in range(p + 1)])

def poly_reg_closed_form(x, y, p):
    """
    Closed-form polynomial regression
    """
    X = poly_design_matrix(x, p)
    theta = np.linalg.pinv(X) @ y
    return theta

def predict_poly(x, theta, p):
    X = poly_design_matrix(x, p)
    return X @ theta


- Consider the house price prediction problem with feature $X=$ `sqft_living`. Train a polynomial regression model for different values of $p \le 5$ using your implementation. Include a table with the MSE and $R^2$ metrics on both the training and testing data for at least 3 different values of $p$. Discuss your observations on how the MSE and $R^2$ metrics change with the degree of the polynomial.

In [20]:
results = []

X_train = train_df[["sqft_living"]]
y_train = train_df["price"] / 1000

X_test = test_df[["sqft_living"]]
y_test = test_df["price"] / 1000

# standardize feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


for p in [1, 2, 3, 5]:
    
    theta = poly_reg_closed_form(X_train, y_train, p)

    y_train_pred = predict_poly(X_train, theta, p)
    y_test_pred  = predict_poly(X_test_scaled, theta, p)


    # metric
    results.append({
        "Degree p": p,
        "Train MSE": mean_squared_error(y_train, y_train_pred),
        "Train R^2": r2_score(y_train, y_train_pred),
        "Test MSE": mean_squared_error(y_test, y_test_pred),
        "Test R^2": r2_score(y_test, y_test_pred)
    })

results_df = pd.DataFrame(results)
print(results_df)


   Degree p     Train MSE  Train R^2       Test MSE  Test R^2
0         1  57947.526161   0.496709   88575.978543  0.468736
1         2  54822.665116   0.523849   71791.679479  0.569406
2         3  53785.194716   0.532860   99833.483763  0.401216
3         5  52626.111955   0.542927  570616.914821 -2.422464


As the polynomial degree increases, the training MSE decreases and the training R² increases, indicating that the model becomes more flexible and fits the training data better. 

However, the testing performance improves only up to degree 2, after which it deteriorates significantly. For degree 5, the testing MSE becomes extremely large and the R² becomes negative, indicating severe overfitting. 

Increasing model complexity reduces bias but increases variance, which harms generalization performance. P=2 provides the best balance between bias and variance for this feature.